<img title="GitHub Octocat" src='./img/Octocat.jpg' style='height: 60px; padding-right: 15px' alt="Octocat" align="left"> This notebook is part of a GitHub repository: https://github.com/pessini/moby-bikes 
<br>MIT Licensed
<br>Author: Leandro Pessini

In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import sys
import os

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Models & Evaluation
from sklearn.model_selection import KFold

# statsmodel
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf
import statsmodels.stats as stats

# Boost models
from xgboost import XGBRegressor
from sklearn import metrics

# Hyperparameter optimization
from bayes_opt import BayesianOptimization

# Custom objects
sys.path.insert(0, os.path.abspath('../src/'))
import experiment_tracker as et

import time
import warnings
warnings.simplefilter('ignore', FutureWarning)
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [36]:
# ideas_df = pd.read_excel('../documentation/experiment_tracker.xlsx', sheet_name='Ideas')
# experiments_df = pd.read_excel('../documentation/experiment_tracker.xlsx', sheet_name='Experiments')

In [37]:
# creates a new object to keep track of the experiments
experiment_tracker = et.ExperimentTracker()

In [38]:
df_train = pd.read_csv('../data/processed/df_train.csv')
df_test = pd.read_csv('../data/processed/df_test.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   rain                8760 non-null   float64
 1   temp                8760 non-null   float64
 2   rhum                8760 non-null   int64  
 3   wdsp                8760 non-null   int64  
 4   date                8760 non-null   object 
 5   hour                8760 non-null   int64  
 6   day                 8760 non-null   int64  
 7   month               8760 non-null   int64  
 8   year                8760 non-null   int64  
 9   count               8760 non-null   int64  
 10  holiday             8760 non-null   bool   
 11  dayofweek_n         8760 non-null   int64  
 12  dayofweek           8760 non-null   object 
 13  working_day         8760 non-null   bool   
 14  season              8760 non-null   object 
 15  peak                8760 non-null   bool   
 16  timeso

In [39]:
df = df_train.copy()
X = df.drop(['count'], axis=1)
y = df.pop('count')
all_columns = list(X.columns)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_val.shape

((6132, 22), (2628, 22))

In [40]:
test_df = df_test.copy()
X_test = test_df.drop(['count'], axis=1)
y_test = test_df.pop('count')
X_test.shape

(1464, 22)

In [41]:
def get_train_val_score(model, predictors, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val):
    
    X_train = X_train[[c for c in X_train.columns if c in predictors]]
    X_val = X_val[[c for c in X_val.columns if c in predictors]]
    model.fit(X_train, y_train)
    
    # Predict on training and validation set
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    
    # Calculate the RSME and MAE
    train_rsme = metrics.mean_squared_error(y_train, y_pred_train, squared=False) 
    # If squared = True returns MSE value, if False returns RMSE value.
    val_rsme = metrics.mean_squared_error(y_val, y_pred_val, squared=False)
    train_mae = metrics.mean_absolute_error(y_train, y_pred_train)
    val_mae = metrics.mean_absolute_error(y_val, y_pred_val)
    
    return train_rsme, val_rsme, train_mae, val_mae

In [42]:
def get_metrics_to_Experiment() -> list:
    rsme = et.Score('RSME', '{:.4f}'.format(train_rsme), '{:.4f}'.format(val_rsme))
    mae = et.Score('MAE', '{:.4f}'.format(train_mae), '{:.4f}'.format(val_mae))
    return [rsme, mae]

In [43]:
import category_encoders as ce

def preprocessor(predictors: list) -> ColumnTransformer:
    # Setting remainder='passthrough' will mean that all columns not specified in the list of “transformers” 
    #   will be passed through without transformation, instead of being dropped

    ##################### Categorical variables #####################
    all_cat_vars = ['timesofday','dayofweek','holiday','peak','hour','working_day','season','month']
    cat_vars = [categorical_var for categorical_var in all_cat_vars if categorical_var in predictors]

    # categorical variables
    cat_pipe = Pipeline([
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])

    cat_encoder = 'cat', cat_pipe, cat_vars

    ##################### Numerical variables #####################
    all_num_vars = ['rain', 'temp', 'rhum','wdsp','temp_r']
    num_vars = [numerical_var for numerical_var in all_num_vars if numerical_var in predictors]

    num_pipe = Pipeline([
        ('scaler', StandardScaler())
        # ('scaler', MinMaxScaler())
    ])

    num_enconder =  'num', num_pipe, num_vars

    ##################### Ordinal variables #####################
    all_ord_vars = ['wind_speed_group','rainfall_intensity']
    ord_vars = [ordinal_var for ordinal_var in all_ord_vars if ordinal_var in predictors]

    ordinal_cols_mapping = []
    if 'wind_speed_group' in predictors:
        ordinal_cols_mapping.append(
            {"col":"wind_speed_group",    
            "mapping": {
                'Calm / Light Breeze': 0, 
                'Breeze': 1, 
                'Moderate Breeze': 2, 
                'Strong Breeze / Near Gale': 3, 
                'Gale / Storm': 4
            }}
        )

    if 'rainfall_intensity' in predictors:
        ordinal_cols_mapping.append(
            {"col":"rainfall_intensity",    
            "mapping": {
                'no rain': 0, 
                'drizzle': 1, 
                'light rain': 2, 
                'moderate rain': 3, 
                'heavy rain': 4
            }}
        )

    # ordinal variables
    ord_pipe = Pipeline([
        ('ordinal', ce.OrdinalEncoder(mapping=ordinal_cols_mapping))
    ])

    ord_enconder =  'ordinal', ord_pipe, ord_vars
    
    #################################################################################
    
    orig_vars = [var for var in predictors if var not in cat_vars and var not in num_vars and var not in ord_vars]
    orig_enconder = 'pass_vars', 'passthrough', orig_vars
     # ['temp_bin','rhum_bin']
    # ord_pipe = 'passthrough'

    transformers_list = []
    transformers_list.append(cat_encoder) if cat_vars else None
    transformers_list.append(ord_enconder) if ord_vars else None
    transformers_list.append(num_enconder) if num_vars else None
    # transformers_list.append(orig_enconder) if orig_vars else None
    
    return ColumnTransformer(transformers=transformers_list, 
                             remainder='drop')

In [44]:
def kfold_score(model, X, y, n_folds=5):
    cv = KFold(n_splits=n_folds, shuffle=True, random_state=2022)
    
    for n_fold, (train_index, test_index) in enumerate(cv.split(X, y)):
        print('#'*40, f'Fold {n_fold+1} out of {cv.n_splits}', '#'*40)
    # scores = []
    # for train_index, test_index in kf.split(X):
    #     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    #     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #     model.fit(X_train, y_train)
    #     y_pred = model.predict(X_test)
    #     scores.append(metrics.mean_squared_error(y_test, y_pred, squared=False))
    # return scores

In [45]:
idea_xgboost = et.Idea(idea='XGBoost', potential_outcome="Baseline XGBoost with all potential features had an excellent train score.")
experiment_tracker.new_idea(idea_xgboost)

predictors = ['temp','rhum','dayofweek', 'holiday','timesofday','wdsp','rainfall_intensity','peak','working_day', 'hour', 'season']

params_xgboost = {'max_depth':10,
                   'seed': 42,
                   'eval_metric': 'rmse',
                   'verbosity': 0
                   }

pipe_xgboost = Pipeline([
    ('preprocessor', preprocessor(predictors)),
    ('model', XGBRegressor(**params_xgboost))
])
train_rsme, val_rsme, train_mae, val_mae = get_train_val_score(pipe_xgboost, predictors)
exp_xgboost = et.Experiment('XGBoost', predictors=predictors, hyperparameters=pipe_xgboost['model'].get_params(),
                               score=get_metrics_to_Experiment(), notes='Baseline XGBoost')
experiment_tracker.add_experiment(exp_xgboost)

--- New Idea added! ---
ID#: 5582097376 
Idea: XGBoost 
Potential Outcome: Baseline XGBoost with all potential features had an excellent train score.

--- New Experiment added! ---
ID#: 5583691840 
Algorithm: XGBoost 
Predictors: ['temp', 'rhum', 'dayofweek', 'holiday', 'timesofday', 'wdsp', 'rainfall_intensity', 'peak', 'working_day', 'hour', 'season']
Hyperparameters: {'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'gpu_id': -1, 'importance_type': 'gain', 'interaction_constraints': '', 'learning_rate': 0.300000012, 'max_delta_step': 0, 'max_depth': 10, 'min_child_weight': 1, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 100, 'n_jobs': 4, 'num_parallel_tree': 1, 'random_state': 42, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'exact', 'validate_parameters': 1, 'verbosity': 0, 'seed': 42, 'eval_metric': 'rmse'}
Date: 15/06

In [46]:
%%time
kfold_score(pipe_xgboost['model'], X_train, y_train)

######################################## Fold 1 out of 5 ########################################
######################################## Fold 2 out of 5 ########################################
######################################## Fold 3 out of 5 ########################################
######################################## Fold 4 out of 5 ########################################
######################################## Fold 5 out of 5 ########################################
CPU times: user 5.64 ms, sys: 3.18 ms, total: 8.81 ms
Wall time: 9.11 ms


In [47]:
# learnings = \
# """XGBoost with almost all default hyperparameters to use as a baseline."""
# experiment_tracker.update_idea(idea_xgboost, learnings=str.strip(learnings))

In [48]:
fdsfsdfsd
experiment_tracker.to_excel('../documentation/experiment_tracker_xgboost.xlsx')

NameError: name 'fdsfsdfsd' is not defined

In [ ]:
import joblib
#save your model or results
joblib.dump(pipe_xgboost['model'], '../models/XGBoost.pkl')

In [ ]:
%reload_ext watermark
%watermark -a "Leandro Pessini" -n -u -v -iv -w

<img title="GitHub Mark" src="./img/GitHub-Mark-64px.png" style="height: 32px; padding-right: 15px" alt="GitHub Mark" align="left"> [GitHub repository](https://github.com/pessini/moby-bikes) <br>Author: Leandro Pessini